Note: This has pricing stats, but as far as I can tell no real sales number.
They have a 0 to 1 score that is supposedly based on sales numbers relative to other objects of the same type.

SeatGeek maintains a comprehensive directory of live events in the United States and Canada (and to a lesser degree, internationally). We created the SeatGeek Platform to enable others to build on top of that data. The SeatGeek Platform makes it easy to construct applications that help users find and discover events by exposing the wealth of data and powerful search behind SeatGeek.com in an easy-to-consume format.

Performers

To find a performer matching your search criteria you will send a request to the /performers endpoint. Sending a request to /performers will return a paginated list of all performers.

All arguments can be used with each other with the exception of the id argument. id can only be used with API-wide arguments, such as those for pagination and sorting.


slug Argument

The slug argument is used to scope the result to a performer matching a slug.

slug: optional, default is null
A query to the slug argument looks like this: /performers?slug=new-york-mets. This will return a performer document matching the New York Mets

Example Requests

New York Mets
$ curl 'http://api.seatgeek.com/2/performers?slug=new-york-mets'

query Argument

The q argument is used to search across all possible parts of a response document.

q: optional, default is null
This is a very broad search, and, like events and venues, will possibly return unexpected results.

Example Requests

Matches 'red hot'
$ curl 'http://api.seatgeek.com/2/performers?q=red+hot'

Matches 'skrillex'
$ curl 'http://api.seatgeek.com/2/performers?q=skrillex'

In [35]:
import anyjson as json
import requests
from bs4 import BeautifulSoup
#import scalpyr
import urllib
from json import load 
import pandas as pd
from pandas import DataFrame
import numpy as np
import codecs
#from urllib import request
from urllib2 import urlopen # python 2.7 version
#import urlopen
from json import load
#import sqlite3
#import csv
#import codecs
#import cStringIO
#import sys
import hashlib

In [36]:
import re
def cleanName(name):
    """
    Use regular expressions to change an artist name string into
    a valid seatgeek slug.
    """
    s = name
    if ', The' in s:
        s = 'The ' + s[:-5]
    r = re.compile(r"\s+")
    s = r.sub("-", s) # "a\nb\nc
    return s.lower()

def searchName(name):
    """
    Use regular expressions to change slug to seach.
    """
    r = re.compile(r"-")
    s = cleanName(name)
    s = r.sub("+", s) # "a\nb\nc
    return s.lower()

# test
print cleanName('Lady Gaga, The')
print searchName('Lady Gaga')

the-lady-gaga
lady+gaga


In [38]:
#ArtistList = DataFrame.from_csv('../mode/mode-report-run-707064319e01.csv',
#                                sep='\t',index_col=False)
ArtistList = pd.DataFrame.from_csv('../billboard/billboard_artist_names_2010_15.csv')
DailyStats = pd.DataFrame()
ArtistList['name_length'] = 0
for i in range(len(ArtistList)):
    ArtistList['name_length'][i] = len(ArtistList['artist_name'][i])
ArtistList = ArtistList[ArtistList['name_length'] <= 60]
ArtistList['artist_name_clean'] = ArtistList.artist_name.apply(cleanName)
ArtistList.head()

KeyError: 'artist_name'

In [4]:
# create performer table
#pfs = pd.DataFrame(columns=pcols)
pfs = pd.DataFrame()

artists = ArtistList.artist_name
#artists = ['Jose Gonzalez','MGMT','Tallest Man on Earth']

for a in artists:

    reader = codecs.getreader("utf-8")
    # slug
    surl = 'http://api.seatgeek.com/2/performers?slug='
    #s = 'jose-gonzalez'
    s = cleanName(a)
    # query
    qurl = 'http://api.seatgeek.com/2/performers?q='
    q = searchName(a)

    # first try the slug
    try:
            url = surl 
            search_term = s

            #url = qurl
            #search_term = q

            url += search_term
            print url

            #response = request.urlopen(url)
            response = urlopen(url)
            json_obj = load(reader(response))
    except:
        pass

    # try searching if it doesn't match the slug
    if len(json_obj['performers']) == 0:
        try:
            url = qurl
            search_term = q

            url += search_term
            print url

            #response = request.urlopen(url)
            response = urlopen(url)
            json_obj = load(reader(response))
        except:
            pass

    pcols = ['id','name','slug','score','short_name','event_count',
             'has_upcoming','genres','type','events','url']

    performers = json_obj['performers']
    for performer in performers:

        try:
            genres = ','.join([g['name'] for g in performer['genres']])
        except:
            genres = ''
        df = pd.DataFrame([[int(performer['id']),performer['name'], 
            performer['slug'], performer['score'],
            performer['short_name'],performer['stats']['event_count'], 
            performer['has_upcoming_events'],genres,performer['type'],'',
            performer['url'] ]],columns=pcols,index=[int(performer['id'])])
        pfs = pfs.append(df)

pfs.head()

http://api.seatgeek.com/2/performers?slug=ke$ha
http://api.seatgeek.com/2/performers?q=ke$ha
http://api.seatgeek.com/2/performers?slug=eminem
http://api.seatgeek.com/2/performers?slug=katy-perry
http://api.seatgeek.com/2/performers?slug=rihanna
http://api.seatgeek.com/2/performers?slug=bruno-mars
http://api.seatgeek.com/2/performers?slug=usher
http://api.seatgeek.com/2/performers?slug=far-east-movement
http://api.seatgeek.com/2/performers?slug=b.o.b-
http://api.seatgeek.com/2/performers?q=b.o.b+
http://api.seatgeek.com/2/performers?slug=the-black-eyed-peas
http://api.seatgeek.com/2/performers?q=the+black+eyed+peas
http://api.seatgeek.com/2/performers?slug=taio-cruz
http://api.seatgeek.com/2/performers?slug=pink
http://api.seatgeek.com/2/performers?slug=lady-antebellum
http://api.seatgeek.com/2/performers?slug=iyaz
http://api.seatgeek.com/2/performers?q=iyaz
http://api.seatgeek.com/2/performers?slug=young-money
http://api.seatgeek.com/2/performers?slug=artists-for-haiti
http://api.seatg

,id,name,slug,score,short_name,event_count,has_upcoming,genres,type,events,url
4169,4169,Kesha,kesha,0.60019,Kesha,0,False,pop,band,,https://seatgeek.com/kesha-tickets
2361,2361,Eminem,eminem,0.78238,Eminem,0,False,rap,band,,https://seatgeek.com/eminem-tickets
1031,1031,Katy Perry,katy-perry,0.76374,Katy Perry,4,True,pop,band,,https://seatgeek.com/katy-perry-tickets
2586,2586,Rihanna,rihanna,0.71411,Rihanna,0,False,pop,band,,https://seatgeek.com/rihanna-tickets
6148,6148,Bruno Mars,bruno-mars,0.75766,Bruno Mars,0,False,pop,band,,https://seatgeek.com/bruno-mars-tickets


In [5]:
json_obj.keys()

[u'meta', u'performers']

In [6]:
# gives a list of different performers
print len(json_obj['performers'])
performers = json_obj['performers']
print [ p['name'] for p in performers]
performer = json_obj['performers'][0]
performer.keys()

0
[]


IndexError: list index out of range

In [9]:
# Create events table
evs = pd.DataFrame()

row_iterator = pfs.iterrows()
#_, last = row_iterator.next()  # take first item from row_iterator
for i, pf in row_iterator:
#if True:
    #print pf
    pid = int(pf['id'])
    #print pid

    # slug
    idurl = 'http://api.seatgeek.com/2/events?performers.id='

    try:
        url = idurl 
        search_term = str(pid)
                
        url += search_term
        print url
        
        #response = request.urlopen(url)
        response = urlopen(url)
        json_obj = load(reader(response))

    except ValueError:
        pass
        #print (ArtistList['artist_name'][a] )
        
    #print json_obj
    
    events = json_obj['events']
    # add list of event ids to performer
    pfs.loc[pid,'events'] = ','.join([ str(e['id']) for e in events ])

    for event in events:
        ecols = []
        efields = []

        # id type and title (and score)
        l = ['id','type','title','short_title','score','url']
        ecols += l
        efields += [event[s] for s in l ]

        # dates and times
        l = ['time_tbd','date_tbd','datetime_local','datetime_utc','created_at']
        ecols += l
        efields += [event[s] for s in l ]

        # performers
        l = ['query_performer','num_performers','performer_ids','performer_slugs']
        perfs = event['performers']
        num_perfs = len(perfs)
        perf_ids = [p['id'] for p in perfs]
        perf_slugs = [p['slug'] for p in perfs]
        ecols += l
        efields += [pid,num_perfs,perf_ids,perf_ids]

        # venue
        l = ['venue_id','venue_slug']
        venue = event['venue']
        venue_id = venue['id'] 
        venue_slug = venue['slug']
        ecols += l
        efields += [venue_id,venue_slug]

        # stats
        stats = event['stats']
        ecols += stats.keys()
        efields += [stats[key] for key in stats.keys()]

        # make dataframe
        ev = pd.DataFrame([efields],columns=ecols, index=[event['id']])
        evs = evs.append(ev)

evs.head()    
    


http://api.seatgeek.com/2/events?performers.id=4169
http://api.seatgeek.com/2/events?performers.id=2361
http://api.seatgeek.com/2/events?performers.id=1031
http://api.seatgeek.com/2/events?performers.id=2586
http://api.seatgeek.com/2/events?performers.id=6148
http://api.seatgeek.com/2/events?performers.id=1969
http://api.seatgeek.com/2/events?performers.id=6332
http://api.seatgeek.com/2/events?performers.id=2272
http://api.seatgeek.com/2/events?performers.id=296963
http://api.seatgeek.com/2/events?performers.id=289118
http://api.seatgeek.com/2/events?performers.id=279175
http://api.seatgeek.com/2/events?performers.id=295518
http://api.seatgeek.com/2/events?performers.id=288972
http://api.seatgeek.com/2/events?performers.id=85
http://api.seatgeek.com/2/events?performers.id=35479
http://api.seatgeek.com/2/events?performers.id=270471
http://api.seatgeek.com/2/events?performers.id=6096
http://api.seatgeek.com/2/events?performers.id=1441
http://api.seatgeek.com/2/events?performers.id=1082
h

,id,type,title,short_title,score,url,time_tbd,date_tbd,datetime_local,datetime_utc,...,num_performers,performer_ids,performer_slugs,venue_id,venue_slug,listing_count,average_price,lowest_price_good_deals,lowest_price,highest_price
2599027,2599027,concert,Katy Perry,Katy Perry,0.7761,https://seatgeek.com/katy-perry-tickets/lima-j...,True,False,2015-09-22T03:30:00,2015-09-22T08:30:00,...,1,[1031],[1031],31784,jockey-club-del-peru,None,None,None,None,None
2590287,2590287,concert,Katy Perry,Katy Perry,0.77647,https://seatgeek.com/katy-perry-tickets/sao-pa...,False,False,2015-09-25T21:00:00,2015-09-26T00:00:00,...,1,[1031],[1031],29937,allianz-parque-1,2,345.5,None,324,367
2658683,2658683,concert,Katy Perry,Katy Perry,0.7761,https://seatgeek.com/katy-perry-tickets/santia...,True,False,2015-10-06T03:30:00,2015-10-06T07:30:00,...,1,[1031],[1031],9869,estadio-nacional,None,None,None,None,None
2599362,2599362,concert,Katy Perry,Katy Perry,0.77638,https://seatgeek.com/katy-perry-tickets/san-ju...,True,False,2015-10-12T03:30:00,2015-10-12T07:30:00,...,1,[1031],[1031],7870,coliseo-de-puerto-rico,None,None,None,None,None
2367207,2367207,concert,Usher,Usher,0.70383,https://seatgeek.com/usher-tickets/vancouver-c...,False,False,2017-11-27T19:30:00,2017-11-28T03:30:00,...,1,[1969],[1969],2983,rogers-arena,None,None,None,None,None


In [ ]:
    pid = 
    #print pid

    # slug
    idurl = 'http://api.seatgeek.com/2/events?performers.id='

    try:
        url = idurl 
        search_term = str(pid)
                
        url += search_term
        print url
        
        #response = request.urlopen(url)
        response = urlopen(url)
        json_obj = load(reader(response))

    except ValueError:
        pass

In [10]:
evs.tail()

,id,type,title,short_title,score,url,time_tbd,date_tbd,datetime_local,datetime_utc,...,num_performers,performer_ids,performer_slugs,venue_id,venue_slug,listing_count,average_price,lowest_price_good_deals,lowest_price,highest_price
2452348,2452348,music_festival,Electric Zoo Festival (3 Day Pass) with David ...,Electric Zoo Festival (3 Day Pass) with David ...,0.67325,https://seatgeek.com/electric-zoo-festival-3-d...,True,False,2015-09-04T03:30:00,2015-09-04T07:30:00,...,80,"[220, 281, 1410, 4114, 4347, 4415, 4862, 4941,...","[220, 281, 1410, 4114, 4347, 4415, 4862, 4941,...",3179,randalls-island,70,403,None,248,613
2636787,2636787,music_festival,Adventure Club (Friday Pass) with Chemical Bro...,Adventure Club (Friday Pass) with Chemical Bro...,0.62844,https://seatgeek.com/adventure-club-friday-pas...,False,False,2015-09-04T11:00:00,2015-09-04T15:00:00,...,70,"[4347, 4862, 6024, 6027, 6177, 6683, 7257, 949...","[4347, 4862, 6024, 6027, 6177, 6683, 7257, 949...",3179,randalls-island,35,187,None,138,540
2652736,2652736,music_festival,Electric Zoo Festival (2 Day Pass) with Chemic...,Electric Zoo Festival (2 Day Pass) with Chemic...,0.65083,https://seatgeek.com/electric-zoo-festival-2-d...,False,False,2015-09-04T13:00:00,2015-09-04T17:00:00,...,23,"[4347, 6024, 6177, 6683, 7257, 9497, 9722, 105...","[4347, 6024, 6177, 6683, 7257, 9497, 9722, 105...",3179,randalls-island,1,225,None,225,225
2636560,2636560,music_festival,"Borgeous with Armin Van Buuren, Borgore, Zeds ...","Borgeous with Armin Van Buuren, Borgore, Zeds ...",0.48856,https://seatgeek.com/borgeous-with-armin-van-b...,False,False,2015-09-05T17:00:00,2015-09-05T23:00:00,...,28,"[220, 9497, 9722, 14274, 16192, 16281, 22532, ...","[220, 9497, 9722, 14274, 16192, 16281, 22532, ...",9895,ascarate-park,1,186,None,186,186
2695528,2695528,music_festival,The Feels Beachside Music & Art Festival with ...,The Feels Beachside Music & Art Festival with ...,0.61636,https://seatgeek.com/the-feels-beachside-music...,False,False,2015-09-18T17:00:00,2015-09-19T00:00:00,...,2,"[195842, 270690]","[195842, 270690]",909,avila-beach-resort,0,None,None,None,None


In [11]:
pfs.head()

,id,name,slug,score,short_name,event_count,has_upcoming,genres,type,events,url
4169,4169,Kesha,kesha,0.60019,Kesha,0,False,pop,band,,https://seatgeek.com/kesha-tickets
2361,2361,Eminem,eminem,0.78238,Eminem,0,False,rap,band,,https://seatgeek.com/eminem-tickets
1031,1031,Katy Perry,katy-perry,0.76374,Katy Perry,4,True,pop,band,"2599027,2590287,2658683,2599362",https://seatgeek.com/katy-perry-tickets
2586,2586,Rihanna,rihanna,0.71411,Rihanna,0,False,pop,band,,https://seatgeek.com/rihanna-tickets
6148,6148,Bruno Mars,bruno-mars,0.75766,Bruno Mars,0,False,pop,band,,https://seatgeek.com/bruno-mars-tickets


In [12]:
# Create venues table
# Example events request
vens = pd.DataFrame()
vurl = 'http://api.seatgeek.com/2/venues/'

# unique venue id
vids = evs.venue_id.unique()

for i, vid in enumerate(vids):
#if True:
    #vid = vids[0]
    
    # slug

    try:
        url = vurl 
        search_term = str(vid)
                
        url += search_term
        print url
        
        #response = request.urlopen(url)
        response = urlopen(url)
        json_obj = load(reader(response))

    except ValueError:
        pass
        #print (ArtistList['artist_name'][a] )
        
    vcols = []
    vfields = []

    # id type and title (and score)
    l = [u'id', u'name',u'country', u'city', u'state', u'extended_address', u'url', 
        u'display_location', u'score', u'postal_code', u'address', u'timezone', u'slug']
    vcols += l
    vfields += [json_obj[key] for key in l]
    
    l = ['event_count','lat','lon']
    vcols += l
    vfields += [json_obj['stats']['event_count'], json_obj['location']['lat'],
                json_obj['location']['lon'] ]
    
    # make dataframe
    ven = pd.DataFrame([vfields],columns=vcols, index=[json_obj['id']])
    vens = vens.append(ven)

vens.head()     

http://api.seatgeek.com/2/venues/31784
http://api.seatgeek.com/2/venues/29937
http://api.seatgeek.com/2/venues/9869
http://api.seatgeek.com/2/venues/7870
http://api.seatgeek.com/2/venues/2983
http://api.seatgeek.com/2/venues/76071
http://api.seatgeek.com/2/venues/7336
http://api.seatgeek.com/2/venues/2176
http://api.seatgeek.com/2/venues/60477
http://api.seatgeek.com/2/venues/1568
http://api.seatgeek.com/2/venues/3703
http://api.seatgeek.com/2/venues/77
http://api.seatgeek.com/2/venues/188
http://api.seatgeek.com/2/venues/698
http://api.seatgeek.com/2/venues/62
http://api.seatgeek.com/2/venues/61
http://api.seatgeek.com/2/venues/72
http://api.seatgeek.com/2/venues/33
http://api.seatgeek.com/2/venues/46
http://api.seatgeek.com/2/venues/163
http://api.seatgeek.com/2/venues/339
http://api.seatgeek.com/2/venues/58670
http://api.seatgeek.com/2/venues/258
http://api.seatgeek.com/2/venues/194
http://api.seatgeek.com/2/venues/336
http://api.seatgeek.com/2/venues/3062
http://api.seatgeek.com/2/

,id,name,country,city,state,extended_address,url,display_location,score,postal_code,address,timezone,slug,event_count,lat,lon
31784,31784,Jockey Club del Peru,Peru,Lima,Lim,"Lima, Peru",https://seatgeek.com/venues/jockey-club-del-pe...,"Lima, Peru",0.00000,45801,El Derby,America/Lima,jockey-club-del-peru,1,-12.0982,-76.9745
29937,29937,Allianz Parque,Brazil,Sao Paulo,None,"Sao Paulo, Brazil",https://seatgeek.com/venues/allianz-parque-1/t...,"Sao Paulo, Brazil",0.00000,05005-000,,America/Sao_Paulo,allianz-parque-1,3,-23.5275,-46.6785
9869,9869,Estadio Nacional,Chile,Santiago,RM,"Santiago, Chile",https://seatgeek.com/venues/estadio-nacional/t...,"Santiago, Chile",0.00000,7780464,,America/Santiago,estadio-nacional,3,-33.4642,-70.6109
7870,7870,Coliseo de Puerto Rico,Puerto Rico,San Juan,PR,"San Juan, Puerto Rico",https://seatgeek.com/venues/coliseo-de-puerto-...,"San Juan, Puerto Rico",0.54185,00918,500 Arterial B St.,America/Puerto_Rico,coliseo-de-puerto-rico,6,18.2016,-66.5813
2983,2983,Rogers Arena,Canada,Vancouver,BC,"Vancouver, Canada",https://seatgeek.com/venues/rogers-arena/tickets/,"Vancouver, Canada",0.76409,V6B 6G1,800 Griffiths Way,America/Vancouver,rogers-arena,62,49.2782,-123.1080


In [13]:
# print to csv files
import time
#ts = time.strftime("%Y%m%d-%H%M%S")
ts ='from_billboard_2010_15'
pfs.to_csv('SeatGeek_performers_'+ts+'.csv',index=False)
evs.to_csv('SeatGeek_events_'+ts+'.csv',index=False)
vens.to_csv('SeatGeek_venues_'+ts+'.csv',index=False)

In [68]:
# songkick (similar information, doesn't seem more useful)

url = 'http://api.songkick.com/api/3.0/artists/mbid:507226f4-0842-4f43-950e-81a7017230ec/gigography.json?apikey=CVpeRf49J3duiKg7&&min_date=2010-01-01&max_date=2015-07-27&per_page=100&page=5'
reader = codecs.getreader("utf-8")

response = urlopen(url)
json_obj = load(response)
print json_obj['resultsPage'].keys()
print json_obj['resultsPage']['totalEntries']
print json_obj['resultsPage']['perPage']
print json_obj['resultsPage']['page']

#event['performance'][0].keys()
#event['performance'][0]['artist']

[u'status', u'totalEntries', u'perPage', u'results', u'page']
485
100
5


In [70]:
print len(json_obj['resultsPage']['results']['event'])
event = json_obj['resultsPage']['results']['event'][84]
event.keys()


85


[u'status',
 u'popularity',
 u'displayName',
 u'ageRestriction',
 u'venue',
 u'uri',
 u'start',
 u'location',
 u'performance',
 u'type',
 u'id']

In [71]:
event

{u'ageRestriction': None,
 u'displayName': u'Zac Brown Band at Amphitheater Northwest (July 12, 2015)',
 u'id': 23157063,
 u'location': {u'city': u'Ridgefield, WA, US',
  u'lat': 45.7441975,
  u'lng': -122.667042},
 u'performance': [{u'artist': {u'displayName': u'Zac Brown Band',
    u'id': 514278,
    u'identifier': [{u'href': u'http://api.songkick.com/api/3.0/artists/mbid:507226f4-0842-4f43-950e-81a7017230ec.json',
      u'mbid': u'507226f4-0842-4f43-950e-81a7017230ec'}],
    u'uri': u'http://www.songkick.com/artists/514278-zac-brown-band?utm_source=35442&utm_medium=partner'},
   u'billing': u'headline',
   u'billingIndex': 1,
   u'displayName': u'Zac Brown Band',
   u'id': 45781813}],
 u'popularity': 0.181577,
 u'start': {u'date': u'2015-07-12',
  u'datetime': u'2015-07-12T19:00:00-0700',
  u'time': u'19:00:00'},
 u'status': u'ok',
 u'type': u'Concert',
 u'uri': u'http://www.songkick.com/concerts/23157063-zac-brown-band-at-amphitheater-northwest?utm_source=35442&utm_medium=partner',

In [8]:
json_obj['resultsPage'].keys()

[u'status', u'totalEntries', u'perPage', u'results', u'page']

In [30]:
json_obj['resultsPage'].keys()


[u'status', u'totalEntries', u'perPage', u'results', u'page']

In [6]:
url = 'http://api.npr.org/list?id=3009&output=JSON&apiKey=MDE5OTYyMjIxMDE0Mzc2OTI5ODJhMTY0Ng001'
reader = codecs.getreader("utf-8")

response = urlopen(url)
json_obj = load(response)

In [ ]:
json_obj['resultsPage']['results']['event'])

In [7]:
json_obj.keys()

[u'typeid', u'subcategory', u'id', u'title']

In [29]:
json_obj[u'subcategory'][27]['item'][1]

{u'id': u'15452537',
 u'num': u'2658',
 u'title': {u'$text': u'Townes Van Zandt'},
 u'type': u'artist'}

In [37]:
alist = []
for i in range(len(json_obj[u'subcategory'])):
    alist += json_obj[u'subcategory'][i]['item']

In [40]:
npr_artist_ids = {a['title']['$text']:a['id'] for a in alist}

In [42]:
npr_artist_ids['The Black Keys']

u'15193815'